# MNIST手写数字识别CNN方法

卷积神经网络的鼻祖LeNet-5，将MNIST数据的识别率提高到了99%，这里我们也自己从头搭建一个卷积神经网络，也可以达到99%的准确率

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.0.1.post2'

# 首先，我们定义一些超参数

In [13]:
BATCH_SIZE=10 #大概需要2G的显存
EPOCHS=1 # 总训练轮数
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多

# 定义训练集
因为Pytorch里面包含了MNIST的数据集，所以我们这里直接使用即可。
如果第一次执行会生成mnist文件夹，并且需要一些时间下载，如果以前下载过就不会再次下载了

由于官方已经实现了dataset，所以这里可以直接使用DataLoader来对数据进行读取

In [14]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

# 定义测试集

In [15]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

# 定义CNN
下面我们定义一个网络，网络包含两个卷积层，conv1和conv2，然后紧接着两个线性层作为输出，最后输出10个维度，这10个维度我们作为0-9的标识来确定识别出的是那个数字

在这里建议大家将每一层的输入和输出维度都作为注释标注出来，这样后面阅读代码的会方便很多

In [16]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

我们实例化一个网络，实例化后使用.to方法将网络移动到GPU

优化器我们也直接选择简单暴力的Adam

In [17]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

下面定义一下训练的函数，我们将训练的所有操作都封装到这个函数中

In [18]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

测试的操作也一样封装成一个函数

In [23]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            print(output.max(1, keepdim=True)[1])
            #print(pred)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

下面开始训练，这里就体现出封装起来的好处了，只要写两行就可以了

In [24]:
for epoch in range(0, EPOCHS):
    #train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

tensor([[9],
        [8],
        [8],
        [3],
        [3],
        [5],
        [7],
        [1],
        [9],
        [8]])
tensor([[0],
        [3],
        [2],
        [7],
        [5],
        [6],
        [5],
        [9],
        [0],
        [7]])
tensor([[5],
        [1],
        [7],
        [7],
        [1],
        [4],
        [7],
        [6],
        [6],
        [3]])
tensor([[2],
        [1],
        [0],
        [4],
        [9],
        [0],
        [7],
        [4],
        [0],
        [1]])
tensor([[8],
        [1],
        [2],
        [7],
        [1],
        [6],
        [0],
        [4],
        [7],
        [2]])
tensor([[4],
        [5],
        [4],
        [3],
        [3],
        [1],
        [2],
        [8],
        [4],
        [4]])
tensor([[7],
        [5],
        [6],
        [1],
        [1],
        [8],
        [9],
        [7],
        [4],
        [1]])
tensor([[1],
        [4],
        [5],
        [9],
        [4],
        [1],
     

tensor([[3],
        [8],
        [5],
        [1],
        [7],
        [8],
        [6],
        [1],
        [0],
        [8]])
tensor([[1],
        [2],
        [9],
        [4],
        [0],
        [4],
        [5],
        [9],
        [2],
        [7]])
tensor([[2],
        [9],
        [0],
        [8],
        [1],
        [9],
        [2],
        [0],
        [3],
        [5]])
tensor([[0],
        [6],
        [7],
        [7],
        [9],
        [9],
        [6],
        [6],
        [1],
        [3]])
tensor([[8],
        [5],
        [7],
        [8],
        [8],
        [8],
        [8],
        [6],
        [8],
        [1]])
tensor([[4],
        [5],
        [4],
        [0],
        [7],
        [0],
        [9],
        [1],
        [9],
        [7]])
tensor([[2],
        [3],
        [8],
        [0],
        [1],
        [7],
        [4],
        [5],
        [8],
        [7]])
tensor([[4],
        [8],
        [0],
        [4],
        [4],
        [0],
     

tensor([[0],
        [5],
        [5],
        [2],
        [9],
        [5],
        [9],
        [2],
        [5],
        [4]])
tensor([[9],
        [0],
        [7],
        [3],
        [4],
        [4],
        [7],
        [2],
        [1],
        [1]])
tensor([[1],
        [6],
        [5],
        [6],
        [9],
        [2],
        [4],
        [5],
        [6],
        [9]])
tensor([[9],
        [2],
        [2],
        [1],
        [6],
        [7],
        [0],
        [7],
        [1],
        [6]])
tensor([[6],
        [9],
        [1],
        [3],
        [1],
        [0],
        [5],
        [4],
        [3],
        [7]])
tensor([[0],
        [7],
        [2],
        [9],
        [1],
        [3],
        [0],
        [7],
        [7],
        [8]])
tensor([[4],
        [5],
        [7],
        [0],
        [9],
        [8],
        [6],
        [4],
        [6],
        [9]])
tensor([[5],
        [5],
        [5],
        [0],
        [1],
        [8],
     

tensor([[4],
        [0],
        [8],
        [4],
        [6],
        [4],
        [8],
        [6],
        [3],
        [0]])
tensor([[5],
        [9],
        [7],
        [8],
        [0],
        [5],
        [2],
        [3],
        [9],
        [6]])
tensor([[4],
        [6],
        [2],
        [4],
        [7],
        [8],
        [2],
        [4],
        [7],
        [8]])
tensor([[9],
        [9],
        [9],
        [6],
        [4],
        [7],
        [8],
        [1],
        [3],
        [2]])
tensor([[9],
        [6],
        [0],
        [9],
        [3],
        [7],
        [0],
        [5],
        [5],
        [7]])
tensor([[8],
        [9],
        [9],
        [1],
        [6],
        [0],
        [4],
        [5],
        [4],
        [6]])
tensor([[8],
        [8],
        [9],
        [8],
        [4],
        [7],
        [5],
        [4],
        [3],
        [9]])
tensor([[2],
        [2],
        [9],
        [6],
        [5],
        [7],
     

tensor([[0],
        [9],
        [4],
        [5],
        [3],
        [1],
        [4],
        [2],
        [9],
        [9]])
tensor([[1],
        [8],
        [7],
        [2],
        [0],
        [7],
        [3],
        [5],
        [7],
        [8]])
tensor([[0],
        [0],
        [8],
        [7],
        [0],
        [4],
        [1],
        [5],
        [3],
        [5]])
tensor([[6],
        [0],
        [5],
        [5],
        [2],
        [7],
        [3],
        [7],
        [9],
        [3]])
tensor([[4],
        [5],
        [4],
        [3],
        [2],
        [3],
        [5],
        [4],
        [0],
        [7]])
tensor([[7],
        [5],
        [5],
        [9],
        [6],
        [2],
        [7],
        [2],
        [6],
        [5]])
tensor([[0],
        [7],
        [1],
        [8],
        [7],
        [1],
        [7],
        [5],
        [6],
        [1]])
tensor([[3],
        [9],
        [3],
        [9],
        [8],
        [6],
     

...

tensor([[5],
        [9],
        [2],
        [8],
        [1],
        [6],
        [5],
        [1],
        [3],
        [0]])
tensor([[7],
        [4],
        [9],
        [8],
        [0],
        [4],
        [1],
        [0],
        [2],
        [4]])
tensor([[9],
        [0],
        [2],
        [0],
        [3],
        [7],
        [3],
        [5],
        [9],
        [2]])
tensor([[5],
        [5],
        [7],
        [9],
        [1],
        [3],
        [2],
        [7],
        [0],
        [1]])
tensor([[6],
        [9],
        [1],
        [0],
        [6],
        [1],
        [6],
        [1],
        [7],
        [3]])
tensor([[0],
        [3],
        [9],
        [5],
        [4],
        [3],
        [1],
        [7],
        [1],
        [4]])
tensor([[8],
        [7],
        [3],
        [0],
        [6],
        [4],
        [8],
        [3],
        [9],
        [4]])
tensor([[7],
        [6],
        [0],
        [7],
        [7],
        [7],
     

tensor([[8],
        [4],
        [3],
        [6],
        [6],
        [7],
        [0],
        [3],
        [7],
        [9]])
tensor([[5],
        [8],
        [9],
        [1],
        [2],
        [5],
        [1],
        [5],
        [5],
        [1]])
tensor([[7],
        [6],
        [7],
        [7],
        [9],
        [2],
        [6],
        [6],
        [5],
        [7]])
tensor([[9],
        [2],
        [0],
        [8],
        [3],
        [1],
        [8],
        [5],
        [6],
        [5]])
tensor([[1],
        [6],
        [6],
        [5],
        [2],
        [6],
        [3],
        [7],
        [6],
        [6]])
tensor([[9],
        [7],
        [0],
        [9],
        [0],
        [7],
        [6],
        [3],
        [9],
        [5]])
tensor([[2],
        [2],
        [0],
        [1],
        [6],
        [4],
        [0],
        [5],
        [9],
        [7]])
tensor([[0],
        [5],
        [4],
        [7],
        [4],
        [5],
     

tensor([[3],
        [7],
        [7],
        [6],
        [7],
        [7],
        [0],
        [1],
        [4],
        [7]])
tensor([[5],
        [1],
        [8],
        [6],
        [0],
        [4],
        [6],
        [2],
        [1],
        [8]])
tensor([[9],
        [9],
        [0],
        [1],
        [0],
        [2],
        [5],
        [6],
        [6],
        [0]])
tensor([[8],
        [1],
        [8],
        [6],
        [4],
        [0],
        [8],
        [7],
        [1],
        [4]])
tensor([[5],
        [1],
        [7],
        [2],
        [5],
        [6],
        [6],
        [4],
        [4],
        [2]])
tensor([[3],
        [0],
        [2],
        [2],
        [8],
        [1],
        [3],
        [2],
        [0],
        [2]])
tensor([[4],
        [9],
        [4],
        [5],
        [7],
        [9],
        [4],
        [0],
        [0],
        [7]])
tensor([[6],
        [8],
        [2],
        [7],
        [8],
        [4],
     

tensor([[3],
        [7],
        [9],
        [2],
        [2],
        [2],
        [3],
        [5],
        [6],
        [8]])
tensor([[8],
        [7],
        [2],
        [3],
        [9],
        [9],
        [3],
        [0],
        [3],
        [7]])
tensor([[6],
        [6],
        [7],
        [8],
        [1],
        [3],
        [3],
        [9],
        [1],
        [8]])
tensor([[8],
        [1],
        [6],
        [1],
        [3],
        [6],
        [3],
        [1],
        [9],
        [1]])
tensor([[3],
        [3],
        [2],
        [1],
        [3],
        [3],
        [7],
        [1],
        [5],
        [1]])
tensor([[3],
        [3],
        [3],
        [0],
        [1],
        [9],
        [5],
        [1],
        [7],
        [4]])
tensor([[1],
        [9],
        [0],
        [7],
        [5],
        [1],
        [1],
        [3],
        [8],
        [9]])
tensor([[6],
        [3],
        [3],
        [4],
        [7],
        [4],
     

tensor([[9],
        [2],
        [6],
        [8],
        [7],
        [6],
        [7],
        [8],
        [9],
        [9]])
tensor([[1],
        [1],
        [3],
        [0],
        [9],
        [2],
        [1],
        [6],
        [3],
        [4]])
tensor([[6],
        [1],
        [3],
        [7],
        [5],
        [4],
        [9],
        [7],
        [2],
        [6]])
tensor([[3],
        [6],
        [8],
        [5],
        [6],
        [2],
        [6],
        [3],
        [9],
        [7]])
tensor([[1],
        [7],
        [2],
        [1],
        [9],
        [4],
        [2],
        [7],
        [3],
        [3]])
tensor([[6],
        [0],
        [7],
        [7],
        [7],
        [5],
        [2],
        [7],
        [4],
        [4]])
tensor([[1],
        [2],
        [0],
        [7],
        [8],
        [1],
        [0],
        [5],
        [2],
        [1]])
tensor([[6],
        [0],
        [0],
        [8],
        [4],
        [7],
     

# 我们看一下结果，仅一轮训练，测试准确率98%，非常强大！

如果你的模型连MNIST都搞不定，那么你的模型没有任何的价值

即使你的模型搞定了MNIST，你的模型也可能没有任何的价值

MNIST是一个很简单的数据集，由于它的局限性只能作为研究用途，对实际应用带来的价值非常有限。但是通过这个例子，我们可以完全了解一个实际项目的工作流程

我们找到数据集，对数据做预处理，定义我们的模型，调整超参数，测试训练，再通过训练结果对超参数进行调整或者对模型进行调整。

并且通过这个实战我们已经有了一个很好的模板，以后的项目都可以以这个模板为样例